In [34]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS")
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df = ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS.get_dataframe()

NAFCUSTOMER_CUSTOMERS_WITH_MORE_THAN_X_REVENUE = dataiku.Dataset("NAFCUSTOMER_CUSTOMERS_WITH_MORE_THAN_X_REVENUE")
NAFCUSTOMER_CUSTOMERS_WITH_MORE_THAN_X_REVENUE_df = NAFCUSTOMER_CUSTOMERS_WITH_MORE_THAN_X_REVENUE.get_dataframe()

# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

/data/dataiku/dss_data/code-envs/python/env_clc/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (6,8,10,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [35]:
df = NAFCUSTOMER_CUSTOMERS_WITH_MORE_THAN_X_REVENUE_df
print(len(df))

133386


In [0]:
# find all customer_names that have other customer names that start with this customer name

import time

idx = 0
unique_customer_names = df['CUSTOMER'].unique()
full_set_n = len(unique_customer_names)

list_customers_ = []
list_potential_matches_ = []

report_every_n = 100
save_every_n = 1000

start = time.time()

for n in unique_customer_names:
    idx+=1
    df_f = df[(df['CUSTOMER'].str.startswith(n, na=False))&(df['CUSTOMER']!=n)]
    match_list = df_f['CUSTOMER'].unique()
    if len(match_list)>0:
        list_customers_.append(n)
        list_potential_matches_.append(match_list)

    if (idx % report_every_n == 0):
        print(idx, "iterations", len(list_customers_), "with potential matches", full_set_n-idx, "remaining", round((idx/full_set_n)*100,2), "% complete")

    if (idx % save_every_n == 0):
        print('SAVING DATAFRAME')
        df_candidates = pd.DataFrame(list_customers_)
        df_candidates.columns = ['CUSTOMER']
        df_candidates['POTENTIAL_MATCHES'] = list_potential_matches_

        ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df = df_candidates

        # Write recipe outputs
        ACCOUNTS_WITH_POTENTIAL_DUPLICATES = dataiku.Dataset("ACCOUNTS_WITH_POTENTIAL_DUPLICATES")
        ACCOUNTS_WITH_POTENTIAL_DUPLICATES.write_with_schema(ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df)
        print(len(ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df), "written")

        end = time.time()
        print(round((end - start)/60,2), "minutes elapsed.")

print(len(list_customers_))

100 iterations 10 with potential matches 133286 remaining 0.07 % complete
200 iterations 23 with potential matches 133186 remaining 0.15 % complete
300 iterations 30 with potential matches 133086 remaining 0.22 % complete
400 iterations 37 with potential matches 132986 remaining 0.3 % complete
500 iterations 44 with potential matches 132886 remaining 0.37 % complete
600 iterations 56 with potential matches 132786 remaining 0.45 % complete
700 iterations 61 with potential matches 132686 remaining 0.52 % complete
800 iterations 66 with potential matches 132586 remaining 0.6 % complete
900 iterations 74 with potential matches 132486 remaining 0.67 % complete
1000 iterations 82 with potential matches 132386 remaining 0.75 % complete
SAVING DATAFRAME
82 rows successfully written (L4dFjXWdVh)
82 written
0.82 minutes elapsed.
1100 iterations 89 with potential matches 132286 remaining 0.82 % complete
1200 iterations 97 with potential matches 132186 remaining 0.9 % complete
1300 iterations 104 

9900 iterations 527 with potential matches 123486 remaining 7.42 % complete
10000 iterations 532 with potential matches 123386 remaining 7.5 % complete
SAVING DATAFRAME
532 rows successfully written (544TeJ6PnL)
532 written
8.21 minutes elapsed.
10100 iterations 536 with potential matches 123286 remaining 7.57 % complete
10200 iterations 541 with potential matches 123186 remaining 7.65 % complete
10300 iterations 543 with potential matches 123086 remaining 7.72 % complete
10400 iterations 548 with potential matches 122986 remaining 7.8 % complete
10500 iterations 555 with potential matches 122886 remaining 7.87 % complete
10600 iterations 557 with potential matches 122786 remaining 7.95 % complete
10700 iterations 563 with potential matches 122686 remaining 8.02 % complete
10800 iterations 566 with potential matches 122586 remaining 8.1 % complete
10900 iterations 569 with potential matches 122486 remaining 8.17 % complete
11000 iterations 576 with potential matches 122386 remaining 8.

19300 iterations 926 with potential matches 114086 remaining 14.47 % complete
19400 iterations 931 with potential matches 113986 remaining 14.54 % complete
19500 iterations 935 with potential matches 113886 remaining 14.62 % complete
19600 iterations 938 with potential matches 113786 remaining 14.69 % complete
19700 iterations 942 with potential matches 113686 remaining 14.77 % complete
19800 iterations 944 with potential matches 113586 remaining 14.84 % complete
19900 iterations 946 with potential matches 113486 remaining 14.92 % complete
20000 iterations 951 with potential matches 113386 remaining 14.99 % complete
SAVING DATAFRAME
951 rows successfully written (r5OZP2vBsp)
951 written
16.39 minutes elapsed.
20100 iterations 955 with potential matches 113286 remaining 15.07 % complete
20200 iterations 957 with potential matches 113186 remaining 15.14 % complete
20300 iterations 959 with potential matches 113086 remaining 15.22 % complete
20400 iterations 965 with potential matches 112